In [1]:
import json
import os
import random
import time
import urllib.parse

import numpy as np
import pandas
import plotly.express as px
import requests
from dotenv import load_dotenv

In [2]:
# トークン読み込み
# .envにMAPBOX_TOKENを入れてください。

load_dotenv()

mapbox_token = os.environ["MAPBOX_TOKEN"]

In [3]:
# 焼きそばのリストをとってくる
# http://www.umya-yakisoba.com/contents/taberu/

yakisoba_dfs = pandas.read_html("http://www.umya-yakisoba.com/contents/taberu/")
print(len(yakisoba_dfs))

5


In [37]:
# 全体を統合して前処理

yakisoba_all = pandas.concat(yakisoba_dfs[2:]).rename(columns={
    "店　名":"店名",
    "住　所":"住所",
    "焼き方":"焼き方:店",
    "焼き方.1":"焼き方:客"
})
# 必要ない列を除去
yakisoba_all = yakisoba_all.drop(columns=["番号", "地図"])
# 住所がNaNなものを除去
yakisoba_all = yakisoba_all.dropna(subset = ["住所"])
# 店名で重複除去
yakisoba_all = yakisoba_all.drop_duplicates(subset=["店名"])

# 列名変更
yakisoba_all = yakisoba_all.reset_index(drop=True)


In [38]:
yakisoba_all

,店名,住所,電話番号（0544）,営業時間（休憩時間含）,休 日,受入人数,駐車場,焼き方:店,焼き方:客,料 金
0,すぎ本（お宮横丁）,宮町4-23,24-8272,10:30-16:00,火曜日,お宮横丁,無,○,NaN,300～600円
1,お食事処岩市,大宮町8-18,27-7153,11:00-20:00,水曜日,28＋座敷7間,2,○,NaN,600-780円
2,ままん,大宮町5-17,29-6288,11:00-14:0018:00-22:00,水・木曜日,26,無,○,NaN,600円
3,みっちゃん家,宝町11-24,27-5305,11:00-15:00(LO14:30),水曜日第１・３日曜日,8＋座敷2間,5,○,NaN,350-600円
4,わびすけ,野中町806,23-8578,11:30-14:0014:30-22:00(LO21:00),水曜日,54,25,○,NaN,620-980円
...,...,...,...,...,...,...,...,...,...,...
147,大勝,淀師136-8,27-3968,10:00－20:00,月曜日,25,7,○,NaN,450～700円
148,うるおいてい,淀師415-2,24-7155,11:30－14:00,月曜日,50,15,○,○,350～800円
149,お好み食堂 伊東,淀師468-2,27-6494,10:30－20:00,月曜、,50,20,○,NaN,350～600円
150,しまい,淀師583-20,22-2767,12:00－22:00,日曜日,15,8,○,NaN,350～550円


In [44]:
yakisoba_all.to_csv("yakisoba_all_tmp.csv",encoding="shift-jis")

UnicodeEncodeError: 'shift_jis' codec can't encode character '\uff5e' in position 56: illegal multibyte sequence

In [41]:
def get_lon_lat(zyuusyo):
    """
    国土地理院のAPIを使って、
    負荷防止のために、ランダムで問い合わせを行うようにしてます。
    
    returnは(lat, lon) として返す
    lon:0
    lat:1
    ref: https://memo.appri.me/programming/gsi-geocoding-api#%E3%82%B8%E3%82%AA%E3%82%B3%E3%83%BC%E3%83%87%E3%82%A3%E3%83%B3%E3%82%B0API
    """
    encoding_zyuusyo = urllib.parse.quote(zyuusyo)
    req_api = requests.get(f"https://msearch.gsi.go.jp/address-search/AddressSearch?q={encoding_zyuusyo}")
    print(req_api.text)
    req_json = json.loads(req_api.text)
    lon_lat = req_json[0]["geometry"]["coordinates"]
    time.sleep(random.randint(2,5))
    return (lon_lat[1], lon_lat[0])

In [42]:
# このコードは実行すると時間がかかるので、LT当日は行わないように

# pandasにlot,latを入れる

# copy_yakisoba_all = yakisoba_all.copy()
# yakisoba_all["lon_lat"] = copy_yakisoba_all["住所"].map(lambda x: get_lon_lat(f"静岡県富士宮市{x}"))
# copy_yakisoba_all = yakisoba_all.copy()
# yakisoba_all["lon"] = copy_yakisoba_all["lon_lat"].map(lambda x: x[1])
# yakisoba_all["lat"] = copy_yakisoba_all["lon_lat"].map(lambda x: x[0])

# INFO:2021-08-26 copyを使うのが微妙に見えてしまう…
# 大体600秒ぐらいかかる

[{"geometry":{"coordinates":[138.610199,35.224991],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"静岡県富士宮市宮町４番２３号"}}]
[{"geometry":{"coordinates":[138.611572,35.224586],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"静岡県富士宮市大宮町８番１８号"}}]
[{"geometry":{"coordinates":[138.611862,35.22504],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"静岡県富士宮市大宮町５番１７号"}}]
[{"geometry":{"coordinates":[138.604813,35.229198],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"静岡県富士宮市宝町１１番２４号"}}]
[{"geometry":{"coordinates":[138.606567,35.216354],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"静岡県富士宮市野中町８０６番地"}}]
[{"geometry":{"coordinates":[138.624725,35.231007],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"静岡県富士宮市東阿幸地２４５番地"}}]
[{"geometry":{"coordinates":[138.616074,35.236752],"type":"Point"},"type":"Feature","properties":{"addressCode":"","title":"静岡県富士宮市ひばりが丘６

In [48]:
yakisoba_all.to_csv("yakisoba_all_loclat.csv",encoding="utf-8")
yakisoba_all.to_csv("yakisoba_all_loclat_sjis.csv",encoding="cp932")

In [50]:
px.set_mapbox_access_token(mapbox_token)

fig = px.scatter_mapbox(yakisoba_all, lat="lat", lon="lon", hover_name="店名",
                zoom=9, title="富士宮焼きそばマップ")
fig.show()

もうちょっときれいにしよう

- ラベルにお店情報が出せるようにする


ここまでのまとめ

- pandas.read_table便利
- 富士宮焼きそばが食べられるお店は中心外以外でもあるが、車がないと行くのが難しい
- mapboxでプロットしたはいいものの、どうやってスマホとかで見ようか悩む

In [ ]:
スマホで見る => Googleマップに載せたい

